In [1]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

c:\Users\z\anaconda3\envs\mlflow\lib\site-packages\hyperopt\atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
# load dataset
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv", sep=";")
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [3]:
#split data into train and test sets
train, test = train_test_split(data, test_size=0.2, random_state=42)
train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
4665,7.3,0.17,0.36,8.20,0.028,44.0,111.0,0.99272,3.14,0.41,12.4,6
1943,6.3,0.25,0.44,11.60,0.041,48.0,195.0,0.99680,3.18,0.52,9.5,5
3399,5.6,0.32,0.33,7.40,0.037,25.0,95.0,0.99268,3.25,0.49,11.1,6
843,6.9,0.19,0.35,1.70,0.036,33.0,101.0,0.99315,3.21,0.54,10.8,7
2580,7.7,0.30,0.26,18.95,0.053,36.0,174.0,0.99976,3.20,0.50,10.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
4426,6.2,0.21,0.52,6.50,0.047,28.0,123.0,0.99418,3.22,0.49,9.9,6
466,7.0,0.14,0.32,9.00,0.039,54.0,141.0,0.99560,3.22,0.43,9.4,6
3092,7.6,0.27,0.52,3.20,0.043,28.0,152.0,0.99129,3.02,0.53,11.4,6
3772,6.3,0.24,0.29,13.70,0.035,53.0,134.0,0.99567,3.17,0.38,10.6,6


In [4]:
train_x = train.drop("quality", axis=1).values
train_y = train["quality"].values.ravel()
test_x = test.drop("quality", axis=1).values
test_y = test["quality"].values.ravel()

In [5]:
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)
signature = infer_signature(train_x, train_y)

In [6]:
train_x.shape

(3134, 11)

In [7]:
# ANN Model
import mlflow.tensorflow


def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    
    #define model architecture
    mean=np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)

    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1)
        ]
    )

    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"], momentum=params["momentum"]),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()]
        )
    
    # train the Ann model with lr and momentum
    with mlflow.start_run(nested=True):
        model.fit(train_x, train_y, epochs=epochs, 
                  validation_data=(valid_x, valid_y), 
                  batch_size=64)
        
        # Evaluate the model
        eval_results = model.evaluate(test_x, test_y, batch_size=64)

        eval_rmse = eval_results[1]

        ## log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        ## log the model
        mlflow.tensorflow.log_model(model, name="model", signature=signature)

        return {"loss":eval_rmse, "status": STATUS_OK, "model": model}

In [8]:
def objective(params):
    #MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
        )
    return result

In [9]:
space = {
    "lr":hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0)
}

In [10]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    #conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials,
    )

    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    ## log the parameters and results
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])

    ## log the model
    mlflow.tensorflow.log_model(best_run["model"], name="model", signature=signature)

    #print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")



Epoch 1/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 12s 270ms/step - loss: 38.2811 - root_mean_squared_error: 6.1872
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 26.9235 - root_mean_squared_error: 5.1566 - val_loss: 7.0254 - val_root_mean_squared_error: 2.6506

Epoch 2/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 5.6855 - root_mean_squared_error: 2.3844
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.7862 - root_mean_squared_error: 2.1822 - val_loss: 2.8607 - val_root_mean_squared_error: 1.6914

Epoch 3/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 2.3860 - root_mean_squared_error: 1.5447
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.4180 - root_mean_squared_error: 1.5544 - val_loss: 2.2854 - val_root_mean_squared_error: 1.5118

 1/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.7869 - root_mean_squared_error: 1.3367
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/ste

In [11]:
from mlflow.models import validate_serving_input
import mlflow.tensorflow

model_uri = "runs:/1a46170767a4491f94216fd4e409f7a6/model"

from mlflow.models import convert_input_example_to_serving_input

serving_payload = convert_input_example_to_serving_input(test_x)

validate_serving_input(model_uri, serving_payload)



c:\Users\z\anaconda3\envs\mlflow\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


array([[ 4.44943  ],
       [ 7.6583314],
       [ 5.8387923],
       [ 4.148024 ],
       [ 5.29726  ],
       [ 7.86415  ],
       [ 5.9823275],
       [ 5.304213 ],
       [ 6.7425594],
       [ 6.2812095],
       [ 6.9844604],
       [ 3.3021345],
       [ 6.55009  ],
       [ 3.9489026],
       [ 5.5895514],
       [ 4.637987 ],
       [ 6.938604 ],
       [ 6.779689 ],
       [ 6.392412 ],
       [ 4.9785886],
       [ 5.502381 ],
       [ 6.017752 ],
       [ 4.3844676],
       [ 5.5367618],
       [ 4.9392614],
       [ 4.045723 ],
       [ 3.7982745],
       [ 5.5045967],
       [ 6.4628854],
       [ 4.2797794],
       [ 5.486789 ],
       [ 4.852791 ],
       [ 6.72616  ],
       [ 4.3293843],
       [ 5.883588 ],
       [ 6.4492455],
       [ 7.080918 ],
       [ 5.3269105],
       [ 5.4158235],
       [ 6.0058722],
       [ 5.166735 ],
       [ 5.143182 ],
       [ 5.1013536],
       [ 6.199927 ],
       [ 4.042072 ],
       [ 5.6916547],
       [ 6.2038817],
       [ 6.32

In [12]:
loaded_model = mlflow.pyfunc.load_model(model_uri)

import pandas as pd
loaded_model.predict(pd.DataFrame(test_x))

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


array([[ 4.44943  ],
       [ 7.6583314],
       [ 5.8387923],
       [ 4.148024 ],
       [ 5.29726  ],
       [ 7.86415  ],
       [ 5.9823275],
       [ 5.304213 ],
       [ 6.7425594],
       [ 6.2812095],
       [ 6.9844604],
       [ 3.3021345],
       [ 6.55009  ],
       [ 3.9489026],
       [ 5.5895514],
       [ 4.637987 ],
       [ 6.938604 ],
       [ 6.779689 ],
       [ 6.392412 ],
       [ 4.9785886],
       [ 5.502381 ],
       [ 6.017752 ],
       [ 4.3844676],
       [ 5.5367618],
       [ 4.9392614],
       [ 4.045723 ],
       [ 3.7982745],
       [ 5.5045967],
       [ 6.4628854],
       [ 4.2797794],
       [ 5.486789 ],
       [ 4.852791 ],
       [ 6.72616  ],
       [ 4.3293843],
       [ 5.883588 ],
       [ 6.4492455],
       [ 7.080918 ],
       [ 5.3269105],
       [ 5.4158235],
       [ 6.0058722],
       [ 5.166735 ],
       [ 5.143182 ],
       [ 5.1013536],
       [ 6.199927 ],
       [ 4.042072 ],
       [ 5.6916547],
       [ 6.2038817],
       [ 6.32

In [13]:
## register in the model registry
mlflow.register_model(model_uri, "wine-quality")

Successfully registered model 'wine-quality'.
2025/07/01 23:20:45 WARNING mlflow.tracking._model_registry.fluent: Run with id 1a46170767a4491f94216fd4e409f7a6 has no artifacts at artifact path 'model', registering model based on models:/m-6b3e319c6ee34cf1b89908fe289ed9f0 instead
Created version '1' of model 'wine-quality'.


<ModelVersion: aliases=[], creation_timestamp=1751383245942, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1751383245942, metrics=[<Metric: dataset_digest=None, dataset_name=None, key='eval_rmse', model_id='m-6b3e319c6ee34cf1b89908fe289ed9f0', run_id='1a46170767a4491f94216fd4e409f7a6', step=0, timestamp=1751382871340, value=1.3375238180160522>], model_id='m-6b3e319c6ee34cf1b89908fe289ed9f0', name='wine-quality', params={'lr': '0.0033801191960781433', 'momentum': '0.16850489202349006'}, run_id='1a46170767a4491f94216fd4e409f7a6', run_link=None, source='models:/m-6b3e319c6ee34cf1b89908fe289ed9f0', status='READY', status_message=None, tags={}, user_id=None, version=1>